In [4]:
import os
import wave
import numpy as np
import pympi.Elan as Elan

def normalize_audio(audio_data):
    return audio_data / np.max(np.abs(audio_data))

def detect_ipus(audio_path, min_sil_dur, min_ipu_dur, silence_percentile):
    with wave.open(audio_path, 'rb') as wf:
        framerate = wf.getframerate()
        nframes = wf.getnframes()
        audio_data = np.frombuffer(wf.readframes(nframes), dtype=np.int16)

    # Normaliser l'audio
    norm_audio = normalize_audio(audio_data)
    abs_audio = np.abs(norm_audio)

    # Définir le seuil basé sur le percentile
    threshold = np.percentile(abs_audio, silence_percentile)
    print(f"Seuil de silence pour {audio_path}: {threshold:.4f}")

    silence_mask = abs_audio < threshold
    sil_start = None
    current_ipu = []
    ipus = []

    for i, silent in enumerate(silence_mask):
        time_sec = i / framerate
        if silent:
            if sil_start is None:
                sil_start = time_sec
        else:
            if sil_start is not None:
                silence_duration = time_sec - sil_start
                if silence_duration >= min_sil_dur:
                    if current_ipu:
                        ipus.append((current_ipu[0], sil_start))
                    current_ipu = [time_sec]
                else:
                    if current_ipu:
                        current_ipu.append(time_sec)
                sil_start = None
            else:
                if not current_ipu:
                    current_ipu = [time_sec]

    # Gérer le dernier IPU
    if current_ipu:
        ipus.append((current_ipu[0], len(audio_data) / framerate))

    # Fusionner les IPUs séparées par des silences inférieurs à min_sil_dur
    merged_ipus = []
    for start, end in ipus:
        if merged_ipus and start - merged_ipus[-1][1] < min_sil_dur:
            merged_ipus[-1] = (merged_ipus[-1][0], end)
        else:
            merged_ipus.append((start, end))

    # Filtrer les IPUs courtes
    filtered_ipus = [(start, end) for start, end in merged_ipus if end - start >= min_ipu_dur]
    return filtered_ipus

def process_eaf_and_audio(eaf_folder, audio_folder, output_folder, min_sil_dur, min_ipu_dur, silence_percentile):
    # Créer le dossier de sortie s'il n'existe pas
    os.makedirs(output_folder, exist_ok=True)

    for eaf_file in os.listdir(eaf_folder):
        if eaf_file.lower().endswith('.eaf'):
            base_name = os.path.splitext(eaf_file)[0]
            audio_file = base_name + '.wav'
            eaf_path = os.path.join(eaf_folder, eaf_file)
            audio_path = os.path.join(audio_folder, audio_file)
            output_eaf_path = os.path.join(output_folder, eaf_file)

            if not os.path.exists(audio_path):
                print(f"Audio correspondant non trouvé pour {eaf_file}")
                continue

            try:
                print(f"\nTraitement de {eaf_file} avec {audio_file}...")

                # Détecter les IPUs
                ipus = detect_ipus(audio_path, min_sil_dur, min_ipu_dur, silence_percentile)
                print(f"IPUs détectées pour {audio_file}: {ipus}")
                print(f"Nombre d'IPUs: {len(ipus)}")

                # Charger le fichier EAF existant
                eaf = Elan.Eaf(eaf_path)

                # Ajouter une nouvelle tier "IPUs" si elle n'existe pas
                tier_name = "IPUs"
                if tier_name not in eaf.get_tier_names():
                    eaf.add_tier(tier_name)

                # Ajouter les annotations des IPUs
                for i, (start, end) in enumerate(ipus):
                    annotation = f"IPU_{i+1}"
                    eaf.add_annotation(tier_name, int(start * 1000), int(end * 1000), annotation)

                # Sauvegarder le fichier EAF modifié dans le dossier de sortie
                eaf.to_file(output_eaf_path)
                print(f"Fichier EAF mis à jour: {output_eaf_path}")

            except Exception as e:
                print(f"Erreur lors du traitement de {eaf_file}: {e}")

if __name__ == "__main__":
    eaf_folder = "/home/or-llsh-156-l01/projets/CREAM/data/Haiti-CMU/annotations/MFA-TextGrid/raw/synth_eaf_pyannote"
    audio_folder = "/home/or-llsh-156-l01/projets/CREAM/data/Haiti-CMU/recordings/converted/synth/wav"
    output_folder = "/home/or-llsh-156-l01/projets/CREAM/data/Haiti-CMU/annotations/MFA-TextGrid/raw/synth_eaf_IPUs"
    min_sil_dur = 0.25
    min_ipu_dur = 0.3
    silence_percentile = 70

    process_eaf_and_audio(eaf_folder, audio_folder, output_folder, min_sil_dur, min_ipu_dur, silence_percentile)



Traitement de bdmw0552.eaf avec bdmw0552.wav...
Seuil de silence pour /home/or-llsh-156-l01/projets/CREAM/data/Haiti-CMU/recordings/converted/synth/wav/bdmw0552.wav: 0.0557
IPUs détectées pour bdmw0552.wav: [(0.14375, 4.74)]
Nombre d'IPUs: 1
Fichier EAF mis à jour: /home/or-llsh-156-l01/projets/CREAM/data/Haiti-CMU/annotations/MFA-TextGrid/raw/synth_eaf_IPUs/bdmw0552.eaf

Traitement de bdmw0070.eaf avec bdmw0070.wav...
Seuil de silence pour /home/or-llsh-156-l01/projets/CREAM/data/Haiti-CMU/recordings/converted/synth/wav/bdmw0070.wav: 0.0376
IPUs détectées pour bdmw0070.wav: [(0.2085, 2.5685625), (3.133125, 3.551), (4.1858125, 4.7050625), (5.2430625, 5.6179375), (5.95375, 6.6600625)]
Nombre d'IPUs: 5
Fichier EAF mis à jour: /home/or-llsh-156-l01/projets/CREAM/data/Haiti-CMU/annotations/MFA-TextGrid/raw/synth_eaf_IPUs/bdmw0070.eaf

Traitement de bdmw1162.eaf avec bdmw1162.wav...
Seuil de silence pour /home/or-llsh-156-l01/projets/CREAM/data/Haiti-CMU/recordings/converted/synth/wav/bdmw